**Importing all the libraries**

In [1]:
import pandas as pd;
import numpy as np;
import os;
import time;
import datetime;
import warnings;
warnings.filterwarnings('ignore');

Assigning path of the folder where **Biz Mix** data files are stored

In [86]:
path = "//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/03. Raw Data/Connect Data/Biz Mix Data"
files = os.listdir(path)

Selecting only **excel files** from all the files available in the above path

In [91]:
files_xls = [f for f in files if (f == "xlsx")]
print("Total number of files in the folder are ",len(files_xls))

Total number of files in the folder are  1


In [92]:
files_xls

['Biz Mix - LDL.xlsx']

**Appending the read files in rows and storing them in a single dataframe** named ***UPC_Monthly***.

Printing number of columns and number of facts in each file.
Also checking the execution time of the code

In [113]:
time_start= time.time()

Biz_Data= pd.DataFrame()

j=0
for f in files_xls:
    
    data = pd.read_excel(path+"/"+f)
    print("\033[1m" + files_xls[j] + "\033[0m",":->\n# of Facts are ",len(set(data["Periods"])),"\t\t # of columns are ",len(data.columns))  
    Biz_Data = pd.concat([Biz_Data,data],axis=0,ignore_index=True)
    j=j+1
    
time_end= time.time();

print("execution time is ", time_end-time_start)

Biz Mix - LDL.xlsx :->
# of Facts are  10 		 # of columns are  11
execution time is  4.842874050140381


In [114]:
# Relaceing in Periods(Facts)
Biz_Data["Periods"]=Biz_Data["Periods"].str.replace("$","temp")
Biz_Data["Periods"]=Biz_Data["Periods"].str.replace("- temp -","-")
Biz_Data["Periods"]=Biz_Data["Periods"].str.replace("temp","$")


Biz_Data["Periods"]=Biz_Data["Periods"].str.replace("CP FORM","CP CATEGORY")
Biz_Data["Periods"]=Biz_Data["Periods"].str.replace("CP USE","CP CATEGORY")
Biz_Data["Periods"]=Biz_Data["Periods"].str.replace("CP LDL AND FOAM","CP CATEGORY")

Biz_Data["Periods"]=Biz_Data["Periods"].str.replace("CP LDL AND FM X DISHAIDS/WPS/SPR","CP CATEGORY")

Biz_Data["Periods"]=Biz_Data["Periods"].str.replace("CP MEGA CATEGORY","CP CATEGORY")
Biz_Data["Periods"]=Biz_Data["Periods"].str.replace("CP BATTERY AND HYBRID","CP CATEGORY")


# Market name replaced
Biz_Data["Markets"]=Biz_Data["Markets"].str.replace("Walgreens Corp Total TA","Walgreens Total TA")

#replacing NaN values with 0
Biz_Data.fillna(value=0,inplace=True)



**Removing unwanted characters from all the column names** and store them in a list ***new_col_names***

In [115]:
##Removing unwanted characters from all the column names** and store them in a list ***new_col_names***

new_col_names=[]
for i in Biz_Data.columns:
    x=len(i)
    if x<=17:
        new_col_names.append(i)
    elif i[:3]=='YTD':
        new_col_names.append(i[:-16].strip('- '))
    else:
        new_col_names.append(i[:-14].strip('- '))
        

## Checking the duplicacte column in** ***Biz_Data***

if(len(new_col_names)!=len(set(new_col_names))):
    print("\033[1;31m"+"*******************************ERROR*******************************\n\t\t\tWe have duplicate columns\n\t\t\tPlease recheck raw data files\n*******************************ERROR*******************************")

##Renaming the column names of ***Biz_Data***

# Biz_Data.rename(columns=dict(zip(Biz_Data.columns,new_col_names)),inplace=True)
Biz_Data.columns=new_col_names

**Adding Missing Quarter**

In [117]:
##**Adding Missing Quater**


# GETTING 20 DAYS BEFORE MONTHS AND YEAR VALUES
now=datetime.datetime.now()
delay= datetime.timedelta(days=20)
a= now - delay
month_number=a.month
year=str(a.year-2)[-2:]
print(month_number)
print(year)

if( month_number >= 1 & month_number <= 3):
    Biz_Data['AMJ '+year]=""
    Biz_Data['JAS '+year]=""
    Biz_Data['OND '+year]=""
elif ( month_number >= 4 & month_number <= 6 ):
    Biz_Data['JAS '+year]=""
    Biz_Data['OND '+year]=""
elif ( month_number >= 7 & month_number <= 9  ):
    Biz_Data['OND '+year]=""

Biz_Data.shape

2
19


(14120, 14)

**Time period mapping**

**Manual Step**

This mapping needs to be changed every year when new year data arrives

In [118]:
##**Time period mapping**



time_period_map= pd.DataFrame({'Connect Time Period':['Markets', 'Products', 'Periods','Cal Yr 2020','Cal Yr 2019','Cal Yr 2018','Cal Yr 2017','YTD','YTD YA','Latest 13 Wks','Latest 26 Wks','OND 21','JAS 21','AMJ 21','JFM 21','OND 20','JAS 20','AMJ 20','JFM 20','OND 19','JAS 19','AMJ 19','Dec 21','Nov 21','Oct 21','Sep 21','Aug 21','Jul 21','Jun 21','May 21','Apr 21','Mar 21','Feb 21','Jan 21','Dec 20','Nov 20','Oct 20','Sep 20','Aug 20','Jul 20','Jun 20','May 20','Apr 20','Mar 20','Feb 20','Jan 20'],
                               'Final Name':['Markets', 'Products', 'Periods','FY 2020','FY 2019','FY 2018','FY 2017','YTD 2021','YTD 2020','Latest 13 Weeks','Latest 26 Weeks','Q4 2021','Q3 2021','Q2 2021','Q1 2021','Q4 2020','Q3 2020','Q2 2020','Q1 2020','Q4 2019','Q3 2019','Q2 2019','Dec 21','Nov 21','Oct 21','Sep 21','Aug 21','Jul 21','Jun 21','May 21','Apr 21','Mar 21','Feb 21','Jan 21','Dec 20','Nov 20','Oct 20','Sep 20','Aug 20','Jul 20','Jun 20','May 20','Apr 20','Mar 20','Feb 20','Jan 20']
                              ,'Order':range(1,47)})

time_period_map.head(5)

##We need to **change column names** of ***Biz_Data*** and **reorder** them based on the time_period_mapping

# assigning variables
LM_column= list(Biz_Data.columns)
time_connect=list(time_period_map["Connect Time Period"])
time_final=list(time_period_map["Final Name"])

# changing column names based on time_period_mapping
new_column=[]
for i in range(0,len(LM_column)):
    for j in range(0,len(time_connect)):
        if LM_column[i]==time_connect[j]:
            new_column.append(time_final[j])
        

print(new_column)

print(len(new_column),len(Biz_Data.columns))

Biz_Data.columns

#assigning new column names
Biz_Data.columns = new_column

# reordering column names
my_list =time_period_map.iloc[:, 1]
my_list1=Biz_Data.columns
result = [x for x in my_list if x in my_list1]
Biz_Data=Biz_Data[result]

['Markets', 'Products', 'Periods', 'Latest 13 Weeks', 'Latest 26 Weeks', 'Feb 21', 'YTD 2021', 'FY 2020', 'FY 2019', 'FY 2018', 'FY 2017', 'Q2 2019', 'Q3 2019', 'Q4 2019']
14 14


**Time Period Mapping**

In [119]:
##**Period Mapping**

Time_Mapping={
    "Nov 19":"4 W/E 11/23/19",
    "Dec 19":"5 W/E 12/28/19",
    "Jan 20":"4 W/E 02/01/20",
    "Feb 20":"4 W/E 02/29/20",
    "Mar 20":"5 W/E 04/04/20",
    "Apr 20":"4 W/E 05/02/20",
    "May 20":"4 W/E 05/30/20",
    "Jun 20":"5 W/E 07/04/20",
    "Jul 20":"4 W/E 08/01/20",
    "Aug 20":"4 W/E 08/29/20",
    "Sep 20":"5 W/E 10/03/20",
    "Oct 20":"4 W/E 10/31/20",
    "Nov 20":"4 W/E 11/28/20",
    "Dec 20":"5 W/E 01/02/21",
    "Jan 21":"4 W/E 01/30/21",
    "Feb 21":"4 W/E 02/27/21",
    "Mar 21":"5 W/E 04/03/21",
    "Apr 21":"4 W/E 05/01/21",
    "May 21":"4 W/E 05/29/21",
    "Jun 21":"5 W/E 07/03/21",
    "Jul 21":"4 W/E 07/31/21",
    "Aug 21":"4 W/E 08/28/21",
    "Sep 21":"5 W/E 10/02/21",
    "Oct 21":"4 W/E 10/30/21",
    "Nov 21":"4 W/E 11/27/21",
    "Dec 21":"5 W/E 01/01/22",
    "Jan 22":"4 W/E 01/29/22",
    "Feb 22":"4 W/E 02/26/22",
    "Mar 22":"5 W/E 04/02/22",
    "Apr 22":"4 W/E 04/30/22",
    "May 22":"5 W/E 05/28/22",
    "Jun 22":"4 W/E 06/25/22",
    "Jul 22":"4 W/E 07/23/22",
    "Aug 22":"5 W/E 08/20/22",
    "Sep 22":"4 W/E 09/17/22",
    "Oct 22":"4 W/E 10/15/22",
    "Nov 22":"5 W/E 11/12/22",
    "Dec 22":"4 W/E 12/10/22",
    "Jan 23":"4 W/E 01/07/23",
    "Feb 23":"5 W/E 02/04/23",
    "Mar 23":"4 W/E 03/04/23",
    "Apr 23":"4 W/E 04/01/23",
    "May 23":"5 W/E 04/29/23",
    "Jun 23":"4 W/E 05/27/23",
    "Jul 23":"4 W/E 06/24/23",
    "Aug 23":"5 W/E 07/22/23",
    "Sep 23":"4 W/E 08/19/23",
    "Oct 23":"5 W/E 09/16/23",
    "Nov 23":"4 W/E 10/14/23",
    "Dec 23":"4 W/E 11/11/23",
    "Jan 24":"5 W/E 12/09/23",
    "Feb 24":"4 W/E 01/06/24",
    "Mar 24":"4 W/E 02/03/24",
    "Apr 24":"5 W/E 03/02/24",
    "May 24":"4 W/E 03/30/24",
    "Jun 24":"4 W/E 04/27/24",
    "Jul 24":"5 W/E 05/25/24",
    "Aug 24":"4 W/E 06/22/24",
    "Sep 24":"4 W/E 07/20/24",
    "Oct 24":"5 W/E 08/17/24",
    "Nov 24":"4 W/E 09/14/24",
    "Dec 24":"4 W/E 10/12/24",
    "Jan 25":"5 W/E 11/09/24",
    "Feb 25":"4 W/E 12/07/24",
    "Mar 25":"5 W/E 01/04/25",
    "Apr 25":"4 W/E 02/01/25",
    "May 25":"4 W/E 03/01/25",
    "Jun 25":"5 W/E 03/29/25",
    "Jul 25":"4 W/E 04/26/25",
    "Aug 25":"4 W/E 05/24/25",
    "Sep 25":"5 W/E 06/21/25",
    "Oct 25":"4 W/E 07/19/25",
    "Nov 25":"4 W/E 08/16/25",
    "Dec 25":"5 W/E 09/13/25",
    "Jan 26":"4 W/E 10/11/25",
    "Feb 26":"4 W/E 11/08/25",
    "Mar 26":"5 W/E 12/06/25",
    "Apr 26":"4 W/E 01/03/26",
    "May 26":"4 W/E 01/31/26",
    "Jun 26":"5 W/E 02/28/26",
    "Jul 26":"4 W/E 03/28/26",
    "Aug 26":"5 W/E 04/25/26",
    "Sep 26":"4 W/E 05/23/26",
    "Oct 26":"4 W/E 06/20/26",
    "Nov 26":"5 W/E 07/18/26",
    "Dec 26":"4 W/E 08/15/26"


}


Biz_Data.rename(columns=Time_Mapping,inplace=True)

**Market MApping**

In [120]:
Market_Mapping={
    
     'ALBSCO Total Company TA':'Alb/Sfy',
     'Agency Grocery':'Agency Grocery',
     'Ahold USA Corp Total TA':"Ahold",
     'BJs Total TA':"BJs",
     'CLUB':"CLUB",
     'CVS Total Corp WO HI TA':"CVS",
     'Dol Gen Total TA':"Dollar General",
     'El Super Total TA':'El Super',
     'FDM':"FDM",
     'Family Dollar Total TA':"Family Dollar",
     'Fiesta Total TA':"Fiesta",
     'Giant Eagle Total TA':"Giant Eagle",
     'Kmart Total TA':"Kmart",
     'Meijer Total TA':"Meijer",
     'Publix Total TA':"Publix",
     'Rite Aid - Corp Total TA':"Rite Aid",
     "Sam's Total TA":"Sams",
     'ShopRite Total TA':"Shoprite",
     'Strategic Grocery':"Strategic Grocery",
     'TOTAL MASS':"TOTAL MASS",
     'Target Total BM/FF TA':"Target",
     'Total US Drug':"US Drug",
     'Total US Food':"US Food",
     'Total US xAOC':"US AOC",
     'VDC':"VDC",
     'Walgreens Total TA':"Walgreens",
     'Walmart Total US TA':"Walmart",
     'Wegmans Total TA':"Wegmans"
    }


Biz_Data["Markets"].replace(Market_Mapping,inplace=True)

**Changing Column Name**

In [121]:
#Changing Column Name

Biz_Data.rename(columns={"Periods":"Facts"},inplace=True)


creating ***movecol*** function to **rearranging** the columns of a dataframe

In [123]:
#***movecol*** function to **rearranging** the columns of a dataframe

def movecol(df, cols_to_move=[], ref_col='', place=''):
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

**Splitting "Products" column**

Creating 3 new columns **"ShareBasis","Char","Manufacturer"** from **"Products"** column by splitting 

In [125]:
Biz_Data[['ShareBasis','Char','Manufacturer']] = Biz_Data['Products'].str.split('|',n=2,expand=True)


Biz_Data[['ShareBasis','Char','Manufacturer']]=Biz_Data[['ShareBasis','Char','Manufacturer']].fillna(value="")
Biz_Data.head(3)

# Moving columns after "Market"
Biz_Data = movecol(Biz_Data, 
             cols_to_move=['ShareBasis','Char','Manufacturer'], 
             ref_col='Markets',
             place='After')
Biz_Data.head(3)

,Markets,ShareBasis,Char,Manufacturer,Products,Facts,FY 2020,FY 2019,FY 2018,FY 2017,YTD 2021,Latest 13 Weeks,Latest 26 Weeks,Q4 2019,Q3 2019,Q2 2019,4 W/E 02/27/21
0,US Drug,LDL/DISHAIDS/WPS/FM/SPRAYS,<8 OZ,,LDL/DISHAIDS/WPS/FM/SPRAYS | <8 OZ,$,1.688855e+07,9935545.059,574.544,499.326,1524557.297,2658429.290,5691225.746,,,,717706.796
1,US Drug,LDL/DISHAIDS/WPS/FM/SPRAYS,<8 OZ,,LDL/DISHAIDS/WPS/FM/SPRAYS | <8 OZ,$ % Chg YA,6.998100e+01,1729192.284,15.064,0.000,-41.263,-28.761,-26.346,,,,-53.182
2,US Drug,LDL/DISHAIDS/WPS/FM/SPRAYS,<8 OZ,,LDL/DISHAIDS/WPS/FM/SPRAYS | <8 OZ,$ Shr - CP CATEGORY,2.159700e+01,16.420,0.001,0.001,16.016,16.628,17.450,,,,15.670


In [128]:
cols=list(Biz_Data.columns[0:4])
Biz_Data[cols]=Biz_Data[cols].apply(lambda x : x.str.strip())

**Facts Mapping**

In [130]:
Biz_Facts_Mapping={
    '$':"Dollars",
     '$ % Chg YA':"Dollars_Chg",
     '$ Shr - CP CATEGORY':"Doll_Shr",
     '$ Shr Chg YA - CP CATEGORY':"Doll_Shr_Chg",
     '%ACV':"Store_selling",
     '%ACV Chg YA':"Store_selling_Chg",
     'Base $ Shr - CP CATEGORY':"Base_Doll_Shr",
     'Base $ Shr Chg YA - CP CATEGORY':"Base_Doll_Shr_Chg",
     'TDP':"Cum_Distri_Pts",
     'TDP Chg YA':"Cum_Distri_Pts_Chg"
}

Biz_Data["Facts"]=Biz_Data["Facts"].replace(Biz_Facts_Mapping)

## Toothpaste

In [ ]:
# path to store Output files
path_TP="//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/5P Output/TP/"

In [ ]:
Biz_Data_TP=Biz_Data[Biz_Data["ShareBasis"]=="TOOTHPASTE"]

## Segment

In [ ]:
Segment= Biz_Data_TP.copy()

Selecting time period and creating filter list for char column

In [ ]:
Char = [ 'ME CAVITY/TC', 'ME PREMIUM MULTI-BENEFIT','ME WHITENING', 'ME ENAMEL', 'ME GUM PROTECTION',
        'ME SENS', 'ME BS/BSPX', 'ME KIDS', 'ME FRESH BREATH', 'ME ADULT VALUE', 'ME DRY MOUTH',
        'ME NATURAL', 'ME MAINSTREAM NATURAL',]

Time_Period=list(Segment.columns[22:23])+list(Segment.columns[12:14])+list(Segment.columns[10:11])+list(Segment.columns[6:10])
print(Time_Period)

Filtering Char Column

In [ ]:
#Filter
Segment=Segment[Segment["Char"].isin(Char)]

Renaming column

In [ ]:
Segment.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_FRANCHISE","Char":"CP_CHARACTERISTICS"
                           },inplace=True)

In [ ]:
Segment["CP_BRAND"]=""
Segment["CP_MANUFACTURER"]="TOOTHPASTE"

In [ ]:
df1=pd.melt(Segment, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)


In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS","PER","Doll_Shr",
                      "Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Dollars",
                      "Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Segment_final=df2[final_column_sorting]

In [ ]:
Segment_final.to_csv(path_TP+"Segment_data.csv",index=False)

In [ ]:
Segment_final.shape

## Price_Tier

In [ ]:
Price_Tier= Biz_Data_TP.copy()

In [ ]:
Char = ["MIDTIER","PREMIUM","SUPER PREMIUM","UBER PREMIUM","ULTRA PREMIUM","VALUE"]
Time_Period=list(Price_Tier.columns[22:23])+list(Price_Tier.columns[12:14])+list(Price_Tier.columns[10:11])+list(Price_Tier.columns[6:10])
print(Time_Period)

In [ ]:
Price_Tier.head()

In [ ]:
#Filter
Price_Tier=Price_Tier[Price_Tier["Char"].isin(Char)]

In [ ]:
Price_Tier.columns

In [ ]:
Price_Tier.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_MANUFACTURER","Char":"CP_CHARACTERISTICS"
                           },inplace=True)

In [ ]:
Price_Tier[["CP_FRANCHISE","CP_BRAND"]]=""

In [ ]:
df1=pd.melt(Price_Tier, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS","PER","Doll_Shr",
                      "Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Dollars",
                      "Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Price_Tier_final=df2[final_column_sorting]

In [ ]:
Price_Tier_final.head(3)

In [ ]:
Price_Tier_final.to_csv(path_TP+"Price_Tier_data.csv",index=False)

## Adult_data

In [ ]:
Adult_data= Biz_Data_TP.copy()

In [ ]:
Char = ["ADULT","KIDS"]
Time_Period=list(Adult_data.columns[22:23])+list(Adult_data.columns[12:14])+list(Adult_data.columns[10:11])+list(Adult_data.columns[6:10])
print(Time_Period)

In [ ]:
#Filter
Adult_data=Adult_data[Adult_data["Char"].isin(Char)]

In [ ]:
Adult_data.columns

In [ ]:
Adult_data.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_MANUFACTURER","Char":"CP_CHARACTERISTICS"
                           },inplace=True)

In [ ]:
Adult_data[["CP_FRANCHISE","CP_BRAND"]]=""

In [ ]:
df1=pd.melt(Adult_data, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS","PER","Doll_Shr",
                      "Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Dollars",
                      "Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Adult_data_final=df2[final_column_sorting]

In [ ]:
Adult_data_final.head(3)

In [ ]:
Adult_data_final.to_csv(path_TP+"Adult_data.csv",index=False)

## MW

In [ ]:
# path to store OuMTBut files
path_MW="//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/5P Output/MW/"

In [ ]:
Biz_Data_MW=Biz_Data[Biz_Data["ShareBasis"]=="MOUTHWASH"]

## Characteristics 3

In [ ]:
Segment= Biz_Data_MW.copy()

In [ ]:
set(Biz_Data_MW["Char"])

In [ ]:
set(Biz_Data_MW[""])

In [ ]:
Segment.columns

In [ ]:
Char = ['ANTISEPTIC','FRESHENING','HERBAL/NATURAL','KIDS/TEENS','MULTI-BENEFIT','ORAL RINSE','SPECIALTY','WHITENING']

Time_Period=list(Segment.columns[16:17])+list(Segment.columns[11:13])+list(Segment.columns[10:11])+list(Segment.columns[6:10])
print(Time_Period)

In [ ]:
#Filter
Segment=Segment[Segment["Char"].isin(Char)]

In [ ]:
Segment.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_MANUFACTURER","Char":"CP_CHARACTERISTIC"
                           },inplace=True)

In [ ]:
Segment.head()

In [ ]:
Segment["CP_BRAND"]=""
Segment["CP_FRANCHISE"]=""

In [ ]:
df1=pd.melt(Segment, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)


In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","PER",
                      "Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg",
                      "Dollars","Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Segment_final=df2[final_column_sorting]

In [ ]:
Segment_final.to_csv(path_MW+"characteristics_3.csv",index=False)

In [ ]:
Segment_final.shape

## Characteristic 1

In [ ]:
Price_Tier= Biz_Data_MW.copy()

In [ ]:
set(Price_Tier["Char"])

In [ ]:
Char = ['HIGH SUPER PREMIUM','LOW SUPER PREMIUM', 'MIDTIER', 'PREMIUM', 'PRVT LBL',
         'SPECIALITY SUPER PREMIUM','VALUE']
Time_Period=list(Price_Tier.columns[16:17])+list(Price_Tier.columns[11:13])+list(Price_Tier.columns[10:11])+list(Price_Tier.columns[6:10])
print(Time_Period)

In [ ]:
Price_Tier.head()

In [ ]:
#Filter
Price_Tier=Price_Tier[Price_Tier["Char"].isin(Char)]

In [ ]:
Price_Tier.columns

In [ ]:
Price_Tier.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_MANUFACTURER","Char":"CP_CHARACTERISTIC"
                           },inplace=True)

In [ ]:
Price_Tier[["CP_FRANCHISE","CP_BRAND"]]=""

In [ ]:
df1=pd.melt(Price_Tier, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","PER","Doll_Shr",
                      "Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Dollars",
                      "Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Price_Tier_final=df2[final_column_sorting]

In [ ]:
Price_Tier_final.head(3)

In [ ]:
Price_Tier_final.to_csv(path_MW+"characteristics_1.csv",index=False)

## Characteristics 2

In [ ]:
Adult_data= Biz_Data_MW.copy()

In [ ]:
Char = ["ADULT","KIDS"]
Time_Period=list(Adult_data.columns[16:17])+list(Adult_data.columns[11:13])+list(Adult_data.columns[10:11])+list(Adult_data.columns[6:10])
print(Time_Period)

In [ ]:
#Filter
Adult_data=Adult_data[Adult_data["Char"].isin(Char)]

In [ ]:
Adult_data.columns

In [ ]:
Adult_data.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_MANUFACTURER","Char":"CP_CHARACTERISTIC"
                           },inplace=True)

In [ ]:
Adult_data[["CP_FRANCHISE","CP_BRAND"]]=""

In [ ]:
df1=pd.melt(Adult_data, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","PER","Doll_Shr",
                      "Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Dollars",
                      "Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Adult_data_final=df2[final_column_sorting]

In [ ]:
Adult_data_final.head(3)

In [ ]:
Adult_data_final.to_csv(path_MW+"characteristics_2.csv",index=False)

## MTB

In [ ]:
# path to store OuMTBut files
path_MTB="//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/5P Output/MTB/"

In [ ]:
Biz_Data_MTB=Biz_Data[Biz_Data["ShareBasis"]=="MANUAL"]

## Characteristics 3

In [ ]:
Segment= Biz_Data_MTB.copy()

In [ ]:
Segment.columns

In [ ]:
Char = ['HIGH PREMIUM', 'HIGH SUPER PREMIUM', 'LOW PREMIUM', 'MIDTIER', 'PREMIUM', 'PRVT LBL',
         'REM PRICE TIER', 'SUPER PREMIUM', 'UBER PREMIUM', 'VALUE']

Time_Period=list(Segment.columns[16:17])+list(Segment.columns[11:13])+list(Segment.columns[10:11])+list(Segment.columns[6:10])
print(Time_Period)

In [ ]:
#Filter
Segment=Segment[Segment["Char"].isin(Char)]

In [ ]:
Segment.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_MANUFACTURER","Char":"CP_CHARACTERISTIC"
                           },inplace=True)

In [ ]:
Segment.head()

In [ ]:
Segment["CP_BRAND"]=""
Segment["CP_FRANCHISE"]=""

In [ ]:
df1=pd.melt(Segment, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)


In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","PER",
                      "Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg",
                      "Dollars","Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Segment_final=df2[final_column_sorting]

In [ ]:
Segment_final.to_csv(path_MTB+"characteristics_3.csv",index=False)

In [ ]:
Segment_final.shape

## Characteristic 1

In [ ]:
Price_Tier= Biz_Data_MTB.copy()

In [ ]:
Char = ['1 PK', '10 PK', '2 PK', '3 PK', '4 PK', '5 PK', '6 PK', '7 PK', '8 PK', '12 PK']
Time_Period=list(Price_Tier.columns[16:17])+list(Price_Tier.columns[11:13])+list(Price_Tier.columns[10:11])+list(Price_Tier.columns[6:10])
print(Time_Period)

In [ ]:
Price_Tier.head()

In [ ]:
#Filter
Price_Tier=Price_Tier[Price_Tier["Char"].isin(Char)]

In [ ]:
Price_Tier.columns

In [ ]:
Price_Tier.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_MANUFACTURER","Char":"CP_CHARACTERISTIC"
                           },inplace=True)

In [ ]:
Price_Tier[["CP_FRANCHISE","CP_BRAND"]]=""

In [ ]:
df1=pd.melt(Price_Tier, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","PER","Doll_Shr",
                      "Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Dollars",
                      "Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Price_Tier_final=df2[final_column_sorting]

In [ ]:
Price_Tier_final.head(3)

In [ ]:
Price_Tier_final.to_csv(path_MTB+"characteristics_1.csv",index=False)

## Characteristics 2

In [ ]:
Adult_data= Biz_Data_MTB.copy()

In [ ]:
Char = ["ADULT","KIDS"]
Time_Period=list(Adult_data.columns[16:17])+list(Adult_data.columns[11:13])+list(Adult_data.columns[10:11])+list(Adult_data.columns[6:10])
print(Time_Period)

In [ ]:
#Filter
Adult_data=Adult_data[Adult_data["Char"].isin(Char)]

In [ ]:
Adult_data.columns

In [ ]:
Adult_data.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_MANUFACTURER","Char":"CP_CHARACTERISTIC"
                           },inplace=True)

In [ ]:
Adult_data[["CP_FRANCHISE","CP_BRAND"]]=""

In [ ]:
df1=pd.melt(Adult_data, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","PER","Doll_Shr",
                      "Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Dollars",
                      "Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Adult_data_final=df2[final_column_sorting]

In [ ]:
Adult_data_final.head(3)

In [ ]:
Adult_data_final.to_csv(path_MTB+"characteristics_2.csv",index=False)

## PTB

In [ ]:
# path to store OuPTBut files
path_PTB="//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/5P Output/PTB/"

In [ ]:
Biz_Data_PTB=Biz_Data[Biz_Data["ShareBasis"].isin(["POWER","BATTERY+HYBRID"])]

In [ ]:
set(Biz_Data_PTB["ShareBasis"])

In [ ]:
set(Biz_Data_PTB["Facts"])

## Characteristic 1

In [ ]:
Price_Tier= Biz_Data_PTB[Biz_Data_PTB["ShareBasis"] == "POWER"]

In [ ]:
set(Price_Tier["Facts"])

In [ ]:
Char = ["ADULT","KIDS"]
Time_Period=list(Price_Tier.columns[16:17])+list(Price_Tier.columns[11:13])+list(Price_Tier.columns[10:11])+list(Price_Tier.columns[6:10])
print(Time_Period)

In [ ]:
Price_Tier.head()

In [ ]:
#Filter
Price_Tier=Price_Tier[Price_Tier["Char"].isin(Char)]

In [ ]:
Price_Tier.columns

In [ ]:
Price_Tier.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_MANUFACTURER","Char":"CP_CHARACTERISTIC"
                           },inplace=True)

In [ ]:
Price_Tier[["CP_FRANCHISE","CP_BRAND"]]=""

In [ ]:
df1=pd.melt(Price_Tier, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","PER","Doll_Shr",
                      "Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Dollars",
                      "Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Price_Tier_final=df2[final_column_sorting]

In [ ]:
Price_Tier_final.head(3)

In [ ]:
Price_Tier_final.to_csv(path_PTB+"characteristics_1.csv",index=False)

## Characteristics 2

In [ ]:
Adult_data= Biz_Data_PTB[Biz_Data_PTB["ShareBasis"]=="BATTERY+HYBRID"]

In [ ]:
set(Adult_data["Facts"])

In [ ]:
Char = ["ADULT","KIDS"]
Time_Period=list(Adult_data.columns[16:17])+list(Adult_data.columns[11:13])+list(Adult_data.columns[10:11])+list(Adult_data.columns[6:10])
print(Time_Period)

In [ ]:
#Filter
Adult_data=Adult_data[Adult_data["Char"].isin(Char)]

In [ ]:
Adult_data.columns

In [ ]:
Adult_data.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_MANUFACTURER","Char":"CP_CHARACTERISTIC"
                           },inplace=True)

In [ ]:
Adult_data[["CP_FRANCHISE","CP_BRAND"]]=""

In [ ]:
df1=pd.melt(Adult_data, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
df2.columns

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","PER","Doll_Shr",
                      "Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Dollars",
                      "Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Adult_data_final=df2[final_column_sorting]

In [ ]:
Adult_data_final.head(3)

In [ ]:
Adult_data_final.to_csv(path_PTB+"characteristics_2.csv",index=False)

## BAR

In [ ]:
# path to store Output files
path_BAR="//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/5P Output/BAR/"

In [ ]:
Biz_Data_BAR=Biz_Data[Biz_Data["ShareBasis"]=="BAR"]


## Characteristics 1

In [ ]:
Characteristics1= Biz_Data_BAR.copy()
Characteristics1.head(1)

In [ ]:
Char = [ "BAR BATH 3.7-6.49 OZ","BAR BATH 3.75-6.49 OZ","BAR FMLY 6.5-9 OZ","BAR PRSNL 2.51-3.69 OZ",
        "BAR SUPER 9.1+ OZ","BAR TRIAL 0-2.5 OZ","REM SIZE RANGE"]

Time_Period=list(Characteristics1.columns[16:17])+list(Characteristics1.columns[11:13])+list(Characteristics1.columns[10:11])+list(Characteristics1.columns[6:10])
print(Time_Period)

In [ ]:
#Filter
Characteristics1=Characteristics1[Characteristics1["Char"].isin(Char)]

In [ ]:
Characteristics1.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_FRANCHISE","Char":"CP_CHARACTERISTICS"
                           },inplace=True)
Characteristics1.head()

In [ ]:
Characteristics1["CP_BRAND"]=""
Characteristics1["CP_MANUFACTURER"]=""

In [ ]:
df1=pd.melt(Characteristics1, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)


In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)
df2.head()

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS","PER","Doll_Shr",
                      "Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Dollars",
                      "Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Characteristics1_final=df2[final_column_sorting]

In [ ]:
Characteristics1_final.to_csv(path_BAR+"Characteristics1.csv",index=False)

In [ ]:
Characteristics1_final.shape

## Characteristics2

In [ ]:
Characteristics2= Biz_Data_BAR.copy()

In [ ]:
Char = ["BASIC FRESHNESS","CLEAN + FRESH","CRAFTED","FEMALE MSTRZNG","FRAGRANCE","GIFT SET",
        "INGREDIENT","MOISTURIZING","MULTI-BENEFIT","NATURAL","REM SEGMENT","REPAIR/RENEW","SENSITIVE/GENTLE CLEANSING"]
Time_Period=list(Characteristics2.columns[16:17])+list(Characteristics2.columns[11:13])+list(Characteristics2.columns[10:11])+list(Characteristics2.columns[6:10])
print(Time_Period)

In [ ]:
Characteristics2.head()

In [ ]:
#Filter
Characteristics2=Characteristics2[Characteristics2["Char"].isin(Char)]

In [ ]:
Characteristics2.columns

In [ ]:
Characteristics2.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_MANUFACTURER","Char":"CP_CHARACTERISTICS"
                           },inplace=True)

In [ ]:
Characteristics2[["CP_FRANCHISE","CP_BRAND"]]=""

In [ ]:
df1=pd.melt(Characteristics2, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS","PER","Doll_Shr",
                      "Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Dollars",
                      "Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Characteristics2_final=df2[final_column_sorting]

In [ ]:
Characteristics2_final.head(3)

In [ ]:
Characteristics2_final.to_csv(path_BAR+"Characteristics2.csv",index=False)

## BW

In [69]:
# path to store OuMTBut files
path_BW="//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/5P Output/BW/"

In [70]:
Biz_Data_BW=Biz_Data[Biz_Data["ShareBasis"]=="BODYWASH"]

In [71]:
set(Biz_Data_BW["ShareBasis"])

{'BODYWASH'}

## Characteristics 3

In [72]:
Segment= Biz_Data_BW.copy()

In [73]:
Segment.columns

Index(['Markets', 'ShareBasis', 'Char', 'Manufacturer', 'Products', 'Facts',
       'FY 2020', 'FY 2019', 'FY 2018', 'FY 2017', 'YTD 2021',
       'Latest 13 Weeks', 'Latest 26 Weeks', 'Q4 2019', 'Q3 2019', 'Q2 2019',
       '4 W/E 02/27/21'],
      dtype='object')

In [74]:
set(Segment["Char"])

{'',
 'AB/GEL 0-1.9 OZ',
 'BAR PRSNL 2.51-3.69 OZ',
 'BAR TRIAL 0-2.5 OZ',
 'BASIC FRESHNESS',
 'BETTER CLEAN',
 'BW 12-14 OZ',
 'BW 14.1-17.9 OZ',
 'BW 18-19.9 OZ',
 'BW 20-23.5 OZ',
 'BW 23.6-24 OZ',
 'BW 24.1 OZ+',
 'BW 3-6.9 OZ',
 'BW 7-11.9 OZ',
 'BW < 3 OZ',
 'BW FOAM',
 'BW KIDS',
 'CLEAN + FRESH',
 'FEMALE',
 'FEMALE BOUTIQUE',
 'FEMALE FRAG',
 'FEMALE MSTRZNG',
 'FEMALE PRVT LBL',
 'FORM EXPANSION',
 'FRAGRANCE',
 'FRAGRANCE STORY',
 'GIFT SET',
 'KIDS',
 'LIQ PMP 11.1-11.9 OZ',
 'LIQ PMP 12-13 OZ',
 'LIQ PMP 16.9 OZ',
 'LIQ PMP 7-7.9 OZ',
 'LIQ PMP 8-8.4 OZ',
 'LIQ PMP < 7 OZ',
 'LIQ RFL 13-25 OZ',
 'MALE',
 'MOISTURIZING',
 'MULTI-FUNCTIONAL',
 'NATURAL',
 'NOT APPLICABLE',
 'REM SIZE RANGE',
 'SENSITIVE/GENTLE CLEANSING'}

In [75]:
Char = ["BASIC FRESHNESS","BETTER CLEAN","BW FOAM","BW KIDS","FEMALE BOUTIQUE","FEMALE FRAG","FEMALE MSTRZNG","FEMALE PRVT LBL","FORM EXPANSION","FRAGRANCE STORY","GIFT SET","MOISTURIZING","MULTI-FUNCTIONAL","NATURAL"]

Time_Period=list(Segment.columns[16:17])+list(Segment.columns[11:13])+list(Segment.columns[10:11])+list(Segment.columns[6:10])
print(Time_Period)

['4 W/E 02/27/21', 'Latest 13 Weeks', 'Latest 26 Weeks', 'YTD 2021', 'FY 2020', 'FY 2019', 'FY 2018', 'FY 2017']


In [76]:
#Filter
Segment=Segment[Segment["Char"].isin(Char)]

In [77]:
Segment.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_MANUFACTURER","Char":"CP_CHARACTERISTIC"
                           },inplace=True)

In [78]:
Segment.head()

,MKT,ShareBasis,CP_CHARACTERISTIC,CP_MANUFACTURER,PROD,Facts,FY 2020,FY 2019,FY 2018,FY 2017,YTD 2021,Latest 13 Weeks,Latest 26 Weeks,Q4 2019,Q3 2019,Q2 2019,4 W/E 02/27/21
10,US AOC,BODYWASH,BASIC FRESHNESS,,BODYWASH | BASIC FRESHNESS,Doll_Shr,10.426,9.766,8.437,8.639,10.825,10.505,10.558,,,,11.461
11,US AOC,BODYWASH,BASIC FRESHNESS,,BODYWASH | BASIC FRESHNESS,Doll_Shr_Chg,0.660,1.329,-0.202,8.639,0.533,0.392,0.425,,,,0.973
12,US AOC,BODYWASH,BASIC FRESHNESS,,BODYWASH | BASIC FRESHNESS,Base_Doll_Shr,9.117,8.374,7.085,7.149,9.280,9.059,9.250,,,,9.744
13,US AOC,BODYWASH,BASIC FRESHNESS,,BODYWASH | BASIC FRESHNESS,Base_Doll_Shr_Chg,0.743,1.289,-0.064,7.149,0.564,0.577,0.510,,,,0.803
14,US AOC,BODYWASH,BASIC FRESHNESS,,BODYWASH | BASIC FRESHNESS,Store_selling,91.548,92.669,90.059,89.094,91.529,91.800,92.025,,,,91.408


In [79]:
Segment["CP_BRAND"]=""
Segment["CP_FRANCHISE"]=""

In [80]:
df1=pd.melt(Segment, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)


In [81]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [82]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [83]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","PER",
                      "Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg",
                      "Dollars","Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Segment_final=df2[final_column_sorting]

In [85]:
Segment_final.to_csv(path_BW+"characteristics_3.csv",index=False)

In [ ]:
Segment_final.shape

## Characteristic 1

In [ ]:
Price_Tier= Biz_Data_BW.copy()

In [ ]:
Char = ["FEMALE","KIDS","MALE"]
Time_Period=list(Price_Tier.columns[16:17])+list(Price_Tier.columns[11:13])+list(Price_Tier.columns[10:11])+list(Price_Tier.columns[6:10])
print(Time_Period)

In [ ]:
Price_Tier.head()

In [ ]:
#Filter
Price_Tier=Price_Tier[Price_Tier["Char"].isin(Char)]

In [ ]:
Price_Tier.columns

In [ ]:
Price_Tier.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_MANUFACTURER","Char":"CP_CHARACTERISTIC"
                           },inplace=True)

In [ ]:
Price_Tier[["CP_FRANCHISE","CP_BRAND"]]=""

In [ ]:
df1=pd.melt(Price_Tier, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","PER","Doll_Shr",
                      "Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Dollars",
                      "Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Price_Tier_final=df2[final_column_sorting]

In [ ]:
Price_Tier_final.head(3)

In [ ]:
Price_Tier_final.to_csv(path_BW+"characteristics_1.csv",index=False)

## Characteristics 2

In [ ]:
Adult_data= Biz_Data_BW.copy()

In [ ]:
set(Adult_data["Char"])

In [ ]:
set(Char)-set(Adult_data["Char"])

In [ ]:
Char = ["REM SIZE RANGE","BW < 3 OZ","BW 12-14 OZ","BW 14.1-17.9 OZ","BW 18-19.9 OZ","BW 20-23.5 OZ","BW 23.6-24 OZ","BW 24.1 OZ+","BW 3-6.9 OZ","BW 7-11.9 OZ"]
Time_Period=list(Adult_data.columns[16:17])+list(Adult_data.columns[11:13])+list(Adult_data.columns[10:11])+list(Adult_data.columns[6:10])
print(Time_Period)

In [ ]:
#Filter
Adult_data=Adult_data[Adult_data["Char"].isin(Char)]

In [ ]:
Adult_data.columns

In [ ]:
Adult_data.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_MANUFACTURER","Char":"CP_CHARACTERISTIC"
                           },inplace=True)

In [ ]:
Adult_data[["CP_FRANCHISE","CP_BRAND"]]=""

In [ ]:
set(Adult_data["Facts"])

In [ ]:
df1=pd.melt(Adult_data, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [ ]:
set(df1["Facts"])

In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
df2.columns

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","PER","Doll_Shr",
                      "Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Dollars",
                      "Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Adult_data_final=df2[final_column_sorting]

In [ ]:
Adult_data_final.head(3)

In [ ]:
Adult_data_final.to_csv(path_BW+"characteristics_2.csv",index=False)

## LHS

In [23]:
# path to store Output files
path_LHS="//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/5P Output/LHS/"

In [24]:
Biz_Data_LHS=Biz_Data[Biz_Data["ShareBasis"]=="LIQUID"]


## Characteristics 1

In [25]:
Characteristics1= Biz_Data_LHS.copy()
Characteristics1.head(1)

,Markets,ShareBasis,Char,Manufacturer,Products,Facts,FY 2020,FY 2019,FY 2018,FY 2017,YTD 2021,Latest 13 Weeks,Latest 26 Weeks,Q4 2019,Q3 2019,Q2 2019,4 W/E 02/27/21
0,US AOC,LIQUID,,,LIQUID,Doll_Shr,100.0,100.0,100.0,100.0,100.0,100.0,100.0,,,,100.0


In [26]:
Char = ["FOAM","LIQUID"]

Time_Period=list(Characteristics1.columns[16:17])+list(Characteristics1.columns[11:13])+list(Characteristics1.columns[10:11])+list(Characteristics1.columns[6:10])
print(Time_Period)

['4 W/E 02/27/21', 'Latest 13 Weeks', 'Latest 26 Weeks', 'YTD 2021', 'FY 2020', 'FY 2019', 'FY 2018', 'FY 2017']


In [27]:
#Filter
Characteristics1=Characteristics1[Characteristics1["Char"].isin(Char)]

In [28]:
Characteristics1.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_FRANCHISE","Char":"CP_CHARACTERISTICS"
                           },inplace=True)
Characteristics1.head()

,MKT,ShareBasis,CP_CHARACTERISTICS,CP_FRANCHISE,PROD,Facts,FY 2020,FY 2019,FY 2018,FY 2017,YTD 2021,Latest 13 Weeks,Latest 26 Weeks,Q4 2019,Q3 2019,Q2 2019,4 W/E 02/27/21
10,US AOC,LIQUID,FOAM,,LIQUID | FOAM,Doll_Shr,21.089,23.274,21.901,20.621,18.980,19.914,19.427,,,,18.697
11,US AOC,LIQUID,FOAM,,LIQUID | FOAM,Doll_Shr_Chg,-2.185,1.373,1.280,20.621,-4.996,-3.611,-3.464,,,,-5.956
12,US AOC,LIQUID,FOAM,,LIQUID | FOAM,Base_Doll_Shr,19.308,20.751,19.539,18.111,18.466,19.178,18.758,,,,18.193
13,US AOC,LIQUID,FOAM,,LIQUID | FOAM,Base_Doll_Shr_Chg,-1.443,1.212,1.428,18.111,-1.361,-0.836,-1.370,,,,-1.847
14,US AOC,LIQUID,FOAM,,LIQUID | FOAM,Store_selling,87.810,91.279,90.903,90.650,88.288,88.094,87.639,,,,88.164


In [29]:
Characteristics1["CP_BRAND"]=""
Characteristics1["CP_MANUFACTURER"]=""

In [30]:
df1=pd.melt(Characteristics1, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)


In [31]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)
df2.head()

,MKT,PROD,CP_MANUFACTURER,CP_FRANCHISE,CP_BRAND,CP_CHARACTERISTICS,PER,Base_Doll_Shr,Base_Doll_Shr_Chg,Cum_Distri_Pts,Cum_Distri_Pts_Chg,Doll_Shr,Doll_Shr_Chg,Dollars,Dollars_Chg,Store_selling,Store_selling_Chg
0,Ahold,LIQUID | FOAM,,,,FOAM,4 W/E 02/27/21,14.083,-3.677,1061.761,-570.701,15.313,-10.963,250528.810,-41.035,99.626,-0.140
1,Ahold,LIQUID | FOAM,,,,FOAM,FY 2017,16.727,16.727,1259.959,1259.959,21.186,21.186,3385139.345,0.000,99.896,99.896
2,Ahold,LIQUID | FOAM,,,,FOAM,FY 2018,17.565,0.838,1399.107,139.148,24.121,2.935,4320464.274,27.630,99.909,0.013
3,Ahold,LIQUID | FOAM,,,,FOAM,FY 2019,18.529,0.964,1653.449,254.342,26.090,1.969,4866153.900,12.630,99.638,-0.271
4,Ahold,LIQUID | FOAM,,,,FOAM,FY 2020,18.422,-0.107,950.567,-702.882,20.011,-6.079,5918553.990,21.627,98.880,-0.758


In [32]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [33]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS","PER","Doll_Shr",
                      "Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Dollars",
                      "Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Characteristics1_final=df2[final_column_sorting]

In [34]:
Characteristics1_final.to_csv(path_LHS+"Characteristics1.csv",index=False)

In [35]:
Characteristics1_final.shape

(2640, 17)

## Characteristics2

In [36]:
Characteristics2= Biz_Data_LHS.copy()

In [37]:
Char = ["PUMP","REFILL"]
Time_Period=list(Characteristics2.columns[16:17])+list(Characteristics2.columns[11:13])+list(Characteristics2.columns[10:11])+list(Characteristics2.columns[6:10])
print(Time_Period)

['4 W/E 02/27/21', 'Latest 13 Weeks', 'Latest 26 Weeks', 'YTD 2021', 'FY 2020', 'FY 2019', 'FY 2018', 'FY 2017']


In [38]:
Characteristics2.head()

,Markets,ShareBasis,Char,Manufacturer,Products,Facts,FY 2020,FY 2019,FY 2018,FY 2017,YTD 2021,Latest 13 Weeks,Latest 26 Weeks,Q4 2019,Q3 2019,Q2 2019,4 W/E 02/27/21
0,US AOC,LIQUID,,,LIQUID,Doll_Shr,100.000,100.000,100.000,100.000,100.000,100.000,100.000,,,,100.000
1,US AOC,LIQUID,,,LIQUID,Doll_Shr_Chg,0.000,0.000,0.000,100.000,0.000,0.000,0.000,,,,0.000
2,US AOC,LIQUID,,,LIQUID,Base_Doll_Shr,89.154,88.993,88.499,86.677,95.155,95.882,95.842,,,,94.855
3,US AOC,LIQUID,,,LIQUID,Base_Doll_Shr_Chg,0.161,0.494,1.822,86.677,10.681,9.049,7.713,,,,12.399
4,US AOC,LIQUID,,,LIQUID,Store_selling,98.056,98.487,98.588,98.628,98.165,98.142,98.122,,,,98.169


In [39]:
#Filter
Characteristics2=Characteristics2[Characteristics2["Char"].isin(Char)]

In [40]:
Characteristics2.columns

Index(['Markets', 'ShareBasis', 'Char', 'Manufacturer', 'Products', 'Facts',
       'FY 2020', 'FY 2019', 'FY 2018', 'FY 2017', 'YTD 2021',
       'Latest 13 Weeks', 'Latest 26 Weeks', 'Q4 2019', 'Q3 2019', 'Q2 2019',
       '4 W/E 02/27/21'],
      dtype='object')

In [41]:
Characteristics2.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_MANUFACTURER","Char":"CP_CHARACTERISTICS"
                           },inplace=True)

In [42]:
Characteristics2[["CP_FRANCHISE","CP_BRAND"]]=""

In [43]:
df1=pd.melt(Characteristics2, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [44]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [45]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [46]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTICS","PER","Doll_Shr",
                      "Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Dollars",
                      "Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Characteristics2_final=df2[final_column_sorting]

In [47]:
Characteristics2_final.head(3)

,MKT,PROD,CP_MANUFACTURER,CP_FRANCHISE,CP_BRAND,CP_CHARACTERISTICS,PER,Doll_Shr,Doll_Shr_Chg,Base_Doll_Shr,Base_Doll_Shr_Chg,Store_selling,Store_selling_Chg,Dollars,Dollars_Chg,Cum_Distri_Pts,Cum_Distri_Pts_Chg
0,Ahold,LIQUID | PUMP,,,,PUMP,4 W/E 02/27/21,70.873,-1.299,63.666,17.243,99.956,0.022,1159514.47,-0.642,4787.720,-199.651
5,Ahold,LIQUID | PUMP,,,,PUMP,Latest 13 Weeks,70.737,-1.358,63.552,13.758,99.973,-0.007,4303694.42,24.590,4742.044,-235.284
6,Ahold,LIQUID | PUMP,,,,PUMP,Latest 26 Weeks,69.839,-2.922,65.561,15.780,99.967,-0.023,9835029.22,40.782,4115.422,-976.758


In [48]:
Characteristics2_final.to_csv(path_LHS+"Characteristics2.csv",index=False)

## LDL

In [132]:
# path to store OuMTBut files
path_LDL="//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/5P Output/LDL/"

In [134]:
Biz_Data_LDL=Biz_Data[Biz_Data["ShareBasis"]=="LDL/DISHAIDS/WPS/FM/SPRAYS"]

In [135]:
set(Biz_Data_LDL["ShareBasis"])

{'LDL/DISHAIDS/WPS/FM/SPRAYS'}

## Characteristics 3

In [136]:
Segment= Biz_Data_LDL.copy()

In [137]:
Segment.columns

Index(['Markets', 'ShareBasis', 'Char', 'Manufacturer', 'Products', 'Facts',
       'FY 2020', 'FY 2019', 'FY 2018', 'FY 2017', 'YTD 2021',
       'Latest 13 Weeks', 'Latest 26 Weeks', 'Q4 2019', 'Q3 2019', 'Q2 2019',
       '4 W/E 02/27/21'],
      dtype='object')

In [138]:
set(Segment["Char"])

{'',
 '16.1-20 OZ',
 '20.1-34 OZ',
 '34.1-44 OZ',
 '44.1-56 OZ',
 '56.1-90 OZ',
 '8-16 OZ',
 '90.1+ OZ',
 '<8 OZ',
 'ANTIBACTERIAL',
 'EFFICACY',
 'FOAM',
 'HANDCARE',
 'NATURAL_ECO',
 'ORIGINAL',
 'REGULAR',
 'REM SEGMENT',
 'REM SIZE RANGE',
 'SENSORIAL',
 'ULTRA'}

In [139]:
Char = ['16.1-20 OZ','20.1-34 OZ','34.1-44 OZ','44.1-56 OZ','56.1-90 OZ','8-16 OZ','90.1+ OZ','<8 OZ','REM SIZE RANGE']

Time_Period=list(Segment.columns[16:17])+list(Segment.columns[11:13])+list(Segment.columns[10:11])+list(Segment.columns[6:10])
print(Time_Period)

['4 W/E 02/27/21', 'Latest 13 Weeks', 'Latest 26 Weeks', 'YTD 2021', 'FY 2020', 'FY 2019', 'FY 2018', 'FY 2017']


In [140]:
#Filter
Segment=Segment[Segment["Char"].isin(Char)]

In [141]:
Segment.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_MANUFACTURER","Char":"CP_CHARACTERISTIC"
                           },inplace=True)

In [142]:
Segment.head()

,MKT,ShareBasis,CP_CHARACTERISTIC,CP_MANUFACTURER,PROD,Facts,FY 2020,FY 2019,FY 2018,FY 2017,YTD 2021,Latest 13 Weeks,Latest 26 Weeks,Q4 2019,Q3 2019,Q2 2019,4 W/E 02/27/21
0,US Drug,LDL/DISHAIDS/WPS/FM/SPRAYS,<8 OZ,,LDL/DISHAIDS/WPS/FM/SPRAYS | <8 OZ,Dollars,1.688855e+07,9935545.059,574.544,499.326,1524557.297,2658429.290,5691225.746,,,,717706.796
1,US Drug,LDL/DISHAIDS/WPS/FM/SPRAYS,<8 OZ,,LDL/DISHAIDS/WPS/FM/SPRAYS | <8 OZ,Dollars_Chg,6.998100e+01,1729192.284,15.064,0.000,-41.263,-28.761,-26.346,,,,-53.182
2,US Drug,LDL/DISHAIDS/WPS/FM/SPRAYS,<8 OZ,,LDL/DISHAIDS/WPS/FM/SPRAYS | <8 OZ,Doll_Shr,2.159700e+01,16.420,0.001,0.001,16.016,16.628,17.450,,,,15.670
3,US Drug,LDL/DISHAIDS/WPS/FM/SPRAYS,<8 OZ,,LDL/DISHAIDS/WPS/FM/SPRAYS | <8 OZ,Doll_Shr_Chg,5.177000e+00,16.419,0.000,0.001,-9.820,-7.458,-8.372,,,,-13.670
4,US Drug,LDL/DISHAIDS/WPS/FM/SPRAYS,<8 OZ,,LDL/DISHAIDS/WPS/FM/SPRAYS | <8 OZ,Base_Doll_Shr,1.007300e+01,9.432,0.001,0.001,6.710,6.906,8.580,,,,6.976


In [143]:
Segment["CP_BRAND"]=""
Segment["CP_FRANCHISE"]=""

In [144]:
df1=pd.melt(Segment, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)


In [145]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [146]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [147]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","PER",
                      "Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg",
                      "Dollars","Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Segment_final=df2[final_column_sorting]

In [148]:
Segment_final.to_csv(path_LDL+"characteristics_3.csv",index=False)

In [149]:
Segment_final.shape

(4744, 17)

## Characteristic 1

In [150]:
Price_Tier= Biz_Data_LDL.copy()

In [151]:
Char = ["FOAM","REGULAR","ULTRA"]
Time_Period=list(Price_Tier.columns[16:17])+list(Price_Tier.columns[11:13])+list(Price_Tier.columns[10:11])+list(Price_Tier.columns[6:10])
print(Time_Period)

['4 W/E 02/27/21', 'Latest 13 Weeks', 'Latest 26 Weeks', 'YTD 2021', 'FY 2020', 'FY 2019', 'FY 2018', 'FY 2017']


In [152]:
Price_Tier.head()

,Markets,ShareBasis,Char,Manufacturer,Products,Facts,FY 2020,FY 2019,FY 2018,FY 2017,YTD 2021,Latest 13 Weeks,Latest 26 Weeks,Q4 2019,Q3 2019,Q2 2019,4 W/E 02/27/21
0,US Drug,LDL/DISHAIDS/WPS/FM/SPRAYS,<8 OZ,,LDL/DISHAIDS/WPS/FM/SPRAYS | <8 OZ,Dollars,1.688855e+07,9935545.059,574.544,499.326,1524557.297,2658429.290,5691225.746,,,,717706.796
1,US Drug,LDL/DISHAIDS/WPS/FM/SPRAYS,<8 OZ,,LDL/DISHAIDS/WPS/FM/SPRAYS | <8 OZ,Dollars_Chg,6.998100e+01,1729192.284,15.064,0.000,-41.263,-28.761,-26.346,,,,-53.182
2,US Drug,LDL/DISHAIDS/WPS/FM/SPRAYS,<8 OZ,,LDL/DISHAIDS/WPS/FM/SPRAYS | <8 OZ,Doll_Shr,2.159700e+01,16.420,0.001,0.001,16.016,16.628,17.450,,,,15.670
3,US Drug,LDL/DISHAIDS/WPS/FM/SPRAYS,<8 OZ,,LDL/DISHAIDS/WPS/FM/SPRAYS | <8 OZ,Doll_Shr_Chg,5.177000e+00,16.419,0.000,0.001,-9.820,-7.458,-8.372,,,,-13.670
4,US Drug,LDL/DISHAIDS/WPS/FM/SPRAYS,<8 OZ,,LDL/DISHAIDS/WPS/FM/SPRAYS | <8 OZ,Base_Doll_Shr,1.007300e+01,9.432,0.001,0.001,6.710,6.906,8.580,,,,6.976


In [153]:
#Filter
Price_Tier=Price_Tier[Price_Tier["Char"].isin(Char)]

In [154]:
Price_Tier.columns

Index(['Markets', 'ShareBasis', 'Char', 'Manufacturer', 'Products', 'Facts',
       'FY 2020', 'FY 2019', 'FY 2018', 'FY 2017', 'YTD 2021',
       'Latest 13 Weeks', 'Latest 26 Weeks', 'Q4 2019', 'Q3 2019', 'Q2 2019',
       '4 W/E 02/27/21'],
      dtype='object')

In [155]:
Price_Tier.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_MANUFACTURER","Char":"CP_CHARACTERISTIC"
                           },inplace=True)

In [156]:
Price_Tier[["CP_FRANCHISE","CP_BRAND"]]=""

In [157]:
df1=pd.melt(Price_Tier, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [158]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [159]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [160]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","PER","Doll_Shr",
                      "Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Dollars",
                      "Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Price_Tier_final=df2[final_column_sorting]

In [161]:
Price_Tier_final.head(3)

,MKT,PROD,CP_MANUFACTURER,CP_FRANCHISE,CP_BRAND,CP_CHARACTERISTIC,PER,Doll_Shr,Doll_Shr_Chg,Base_Doll_Shr,Base_Doll_Shr_Chg,Store_selling,Store_selling_Chg,Dollars,Dollars_Chg,Cum_Distri_Pts,Cum_Distri_Pts_Chg
0,Ahold,LDL/DISHAIDS/WPS/FM/SPRAYS | FOAM,,,,FOAM,4 W/E 02/27/21,1.394,-0.500,1.516,-0.271,85.586,-5.151,35568.19,-25.765,95.758,-8.232
5,Ahold,LDL/DISHAIDS/WPS/FM/SPRAYS | FOAM,,,,FOAM,Latest 13 Weeks,1.650,-0.031,1.688,0.086,86.729,-1.873,142288.84,3.560,99.524,-3.879
6,Ahold,LDL/DISHAIDS/WPS/FM/SPRAYS | FOAM,,,,FOAM,Latest 26 Weeks,1.841,0.249,1.772,0.245,84.778,-0.974,319931.51,28.610,97.107,-4.190


In [162]:
Price_Tier_final.to_csv(path_LDL+"characteristics_1.csv",index=False)

## Characteristics 2

In [163]:
Adult_data= Biz_Data_LDL.copy()

In [164]:
set(Adult_data["Char"])

{'',
 '16.1-20 OZ',
 '20.1-34 OZ',
 '34.1-44 OZ',
 '44.1-56 OZ',
 '56.1-90 OZ',
 '8-16 OZ',
 '90.1+ OZ',
 '<8 OZ',
 'ANTIBACTERIAL',
 'EFFICACY',
 'FOAM',
 'HANDCARE',
 'NATURAL_ECO',
 'ORIGINAL',
 'REGULAR',
 'REM SEGMENT',
 'REM SIZE RANGE',
 'SENSORIAL',
 'ULTRA'}

In [165]:
set(Char)-set(Adult_data["Char"])

set()

In [166]:
Char = ['ANTIBACTERIAL','EFFICACY','HANDCARE','NATURAL_ECO','ORIGINAL','REM SEGMENT','SENSORIAL']
Time_Period=list(Adult_data.columns[16:17])+list(Adult_data.columns[11:13])+list(Adult_data.columns[10:11])+list(Adult_data.columns[6:10])
print(Time_Period)

['4 W/E 02/27/21', 'Latest 13 Weeks', 'Latest 26 Weeks', 'YTD 2021', 'FY 2020', 'FY 2019', 'FY 2018', 'FY 2017']


In [167]:
#Filter
Adult_data=Adult_data[Adult_data["Char"].isin(Char)]

In [169]:
Adult_data.columns

Index(['Markets', 'ShareBasis', 'Char', 'Manufacturer', 'Products', 'Facts',
       'FY 2020', 'FY 2019', 'FY 2018', 'FY 2017', 'YTD 2021',
       'Latest 13 Weeks', 'Latest 26 Weeks', 'Q4 2019', 'Q3 2019', 'Q2 2019',
       '4 W/E 02/27/21'],
      dtype='object')

In [170]:
Adult_data.rename(columns={"Markets":"MKT","Products":"PROD","Manufacturer":"CP_MANUFACTURER","Char":"CP_CHARACTERISTIC"
                           },inplace=True)

In [171]:
Adult_data[["CP_FRANCHISE","CP_BRAND"]]=""

In [172]:
set(Adult_data["Facts"])

{'Base_Doll_Shr',
 'Base_Doll_Shr_Chg',
 'Cum_Distri_Pts',
 'Cum_Distri_Pts_Chg',
 'Doll_Shr',
 'Doll_Shr_Chg',
 'Dollars',
 'Dollars_Chg',
 'Store_selling',
 'Store_selling_Chg'}

In [173]:
df1=pd.melt(Adult_data, id_vars =["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [174]:
set(df1["Facts"])

{'Base_Doll_Shr',
 'Base_Doll_Shr_Chg',
 'Cum_Distri_Pts',
 'Cum_Distri_Pts_Chg',
 'Doll_Shr',
 'Doll_Shr_Chg',
 'Dollars',
 'Dollars_Chg',
 'Store_selling',
 'Store_selling_Chg'}

In [175]:
df2=df1.pivot_table(index=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [176]:
df2.columns

Index(['MKT', 'PROD', 'CP_MANUFACTURER', 'CP_FRANCHISE', 'CP_BRAND',
       'CP_CHARACTERISTIC', 'PER', 'Base_Doll_Shr', 'Base_Doll_Shr_Chg',
       'Cum_Distri_Pts', 'Cum_Distri_Pts_Chg', 'Doll_Shr', 'Doll_Shr_Chg',
       'Dollars', 'Dollars_Chg', 'Store_selling', 'Store_selling_Chg'],
      dtype='object')

In [177]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [178]:
final_column_sorting=["MKT","PROD","CP_MANUFACTURER","CP_FRANCHISE","CP_BRAND","CP_CHARACTERISTIC","PER","Doll_Shr",
                      "Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Dollars",
                      "Dollars_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg"
                      ]
Adult_data_final=df2[final_column_sorting]

In [179]:
Adult_data_final.head(3)

,MKT,PROD,CP_MANUFACTURER,CP_FRANCHISE,CP_BRAND,CP_CHARACTERISTIC,PER,Doll_Shr,Doll_Shr_Chg,Base_Doll_Shr,Base_Doll_Shr_Chg,Store_selling,Store_selling_Chg,Dollars,Dollars_Chg,Cum_Distri_Pts,Cum_Distri_Pts_Chg
0,Ahold,LDL/DISHAIDS/WPS/FM/SPRAYS | ANTIBACTERIAL,,,,ANTIBACTERIAL,4 W/E 02/27/21,14.456,0.201,13.464,2.312,99.780,0.089,368719.77,2.229,735.936,-77.006
5,Ahold,LDL/DISHAIDS/WPS/FM/SPRAYS | ANTIBACTERIAL,,,,ANTIBACTERIAL,Latest 13 Weeks,13.553,-0.783,12.348,0.626,99.780,-0.018,1168602.02,-0.264,618.048,-223.583
6,Ahold,LDL/DISHAIDS/WPS/FM/SPRAYS | ANTIBACTERIAL,,,,ANTIBACTERIAL,Latest 26 Weeks,12.320,-2.192,11.403,-0.376,99.745,-0.101,2140731.75,-5.583,550.234,-295.011


In [180]:
Adult_data_final.to_csv(path_LDL+"characteristics_2.csv",index=False)